In [228]:
'''
Name: Ananya Singh
Class: CS677
Date: 12/02/2024
Kaggle Competetion Submission
'''

'\nName: Ananya Singh\nClass: CS677\nDate: 12/02/2024\nKaggle Competetion Submission\n'

In [229]:
'''
*** NOTE FOR GRADER ****
A lot of the work here is based on the live lectures where the professor has provided his code examples/hints where I try to leverage/tweak around with it
'''

'\n*** NOTE FOR GRADER ****\nA lot of the work here is based on the live lectures where the professor has provided his code examples/hints where I try to leverage/tweak around with it\n'

In [230]:
import pandas as pd

In [231]:
df = pd.read_csv('train_data.csv')

In [232]:
test_features = pd.read_csv('test_data.csv')

In [233]:
df

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed,radiant_win,ID
0,871,22,0,4,2,110,2,3,11,3,...,0,10,1,0.875000,0,0,6,3,True,a363534a6344f1b0be1d7ba2c4047d9a
1,2549,22,0,17,0,114,16,2,12,24,...,1,10,0,0.535714,1,0,0,0,True,a0ba4ef0965f56d2eba69c2b9ef33353
2,1841,22,0,8,1,100,2,11,12,2,...,0,13,0,0.727273,0,0,0,0,True,18873e56c2142af326b4e08ca41df63a
3,2211,22,7,11,3,32,14,3,11,21,...,0,1,0,0.347826,0,0,0,0,True,c143931a6a8b3fb55a8ef6b9f30c6933
4,458,22,7,1,0,68,3,0,0,15,...,0,6,0,0.000000,0,0,1,1,True,5a324d8b37522e9f9684493465720023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29670,1664,3,0,8,0,17,1,6,1,8,...,0,4,0,0.363636,1,0,0,0,False,db7050396622d08c6e8a5d6aeedf5f44
29671,2898,22,7,26,108,89,4,5,17,5,...,0,7,0,0.476190,0,0,0,0,True,b1259dbd78c0e36cba45c2f567c9d3de
29672,1246,23,0,6,10,51,9,7,15,16,...,0,3,0,0.433333,0,0,0,0,True,9bae67d14d950856a9693b7efd9bce3f
29673,2620,22,7,15,10,114,9,8,5,10,...,0,11,0,0.780488,1,0,0,0,False,de2e339479cf8e5697aff693d190e411


In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29675 entries, 0 to 29674
Columns: 247 entries, game_time to ID
dtypes: bool(1), float64(30), int64(215), object(1)
memory usage: 55.7+ MB


In [235]:
# df[[x for x in df.columns if 'rosh' in x]]
df['radient_killed_roshan'] = df[['r1_roshans_killed',
   'r2_roshans_killed',
   'r3_roshans_killed',
   'r4_roshans_killed',
   'r5_roshans_killed'
]].sum(axis=1)

df['radient_killed_roshan']

0        0
1        1
2        0
3        0
4        0
        ..
29670    0
29671    2
29672    0
29673    0
29674    0
Name: radient_killed_roshan, Length: 29675, dtype: int64

In [236]:
test_features['radient_killed_roshan'] = test_features[[
    'r1_roshans_killed',
    'r2_roshans_killed',
    'r3_roshans_killed',
    'r4_roshans_killed',
    'r5_roshans_killed'
]].sum(axis=1)

In [56]:
# X_train, X_val = train_test_split(df, test_size=0.2)

In [ ]:
# y_train = X_train['radiant_win']
# X_train = X_train.drop(['radiant_win', 'ID'], axis=1)

# y_val = X_val['radiant_win']
# X_val = X_val.drop(['radiant_win', 'ID'], axis=1)


# X_test = test_features.drop('ID', axis=1)

In [237]:
# Feature Engineering  from lecture 4

# Professor has stated that FIRST BLOOD is a pretty big signal and add_hero_features() method will be VERY helpful for feature engineering

# Below are methods professor created
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
'''

def safe_divide(a, b):
    """Safe division for pandas Series that handles zeros"""
    return pd.Series(np.divide(a, b, where=b != 0)).fillna(0)

def add_lane_dominance_features(df):
    for i in range(6):
        r_prefix = f"r{i}"
        d_prefix = f"d{i}"

        # Last hit efficiency
        df[f"{r_prefix}_lane_control"] = df[f"{r_prefix}_lh"] / (df[f"{r_prefix}_denies"] + 1)
        df[f"{d_prefix}_lane_control"] = df[f"{d_prefix}_lh"] / (df[f"{d_prefix}_denies"] + 1)

        # Gold per minute
        minutes = df["game_time"].clip(lower=60) / 60  # minimum 1 minute
        df[f"{r_prefix}_gold_per_min"] = df[f"{r_prefix}_gold"] / minutes
        df[f"{d_prefix}_gold_per_min"] = df[f"{d_prefix}_gold"] / minutes

        # KDA
        df[f"{r_prefix}_kda"] = (df[f"{r_prefix}_kills"] + df[f"{r_prefix}_assists"]) / (df[f"{r_prefix}_deaths"] + 1)
        df[f"{d_prefix}_kda"] = (df[f"{d_prefix}_kills"] + df[f"{d_prefix}_assists"]) / (df[f"{d_prefix}_deaths"] + 1)

    return df

'''
def add_team_features(df):
    # Radiant team totals
    r_kills = df[[f"r{i}_kills" for i in range(1, 6)]].sum(axis=1)
    r_deaths = df[[f"r{i}_deaths" for i in range(1, 6)]].sum(axis=1)
    r_assists = df[[f"r{i}_assists" for i in range(1, 6)]].sum(axis=1)
    r_gold = df[[f"r{i}_gold" for i in range(1, 6)]].sum(axis=1)
    r_xp = df[[f"r{i}_xp" for i in range(1, 6)]].sum(axis=1)

    # Dire team totals
    d_kills = df[[f"d{i}_kills" for i in range(1, 6)]].sum(axis=1)
    d_deaths = df[[f"d{i}_deaths" for i in range(1, 6)]].sum(axis=1)
    d_assists = df[[f"d{i}_assists" for i in range(1, 6)]].sum(axis=1)
    d_gold = df[[f"d{i}_gold" for i in range(1, 6)]].sum(axis=1)
    d_xp = df[[f"d{i}_xp" for i in range(1, 6)]].sum(axis=1)

    # Team KDAs
    df["radiant_team_kda"] = (r_kills + r_assists) / (r_deaths + 1)
    df["dire_team_kda"] = (d_kills + d_assists) / (d_deaths + 1)

    # Resource advantages (normalized)
    total_gold = r_gold + d_gold
    total_xp = r_xp + d_xp

    df["gold_advantage"] = (r_gold - d_gold) / total_gold.replace(0, 1)
    df["xp_advantage"] = (r_xp - d_xp) / total_xp.replace(0, 1)
    df["kill_advantage"] = r_kills - d_kills

    return df
'''

def add_objective_features(df):
    # Vision control
    r_obs = df[[f"r{i}_obs_placed" for i in range(1, 6)]].sum(axis=1)
    r_sen = df[[f"r{i}_sen_placed" for i in range(1, 6)]].sum(axis=1)
    d_obs = df[[f"d{i}_obs_placed" for i in range(1, 6)]].sum(axis=1)
    d_sen = df[[f"d{i}_sen_placed" for i in range(1, 6)]].sum(axis=1)

    df["vision_advantage"] = (r_obs + r_sen) - (d_obs + d_sen)

    # Tower control
    r_towers = df[[f"r{i}_towers_killed" for i in range(1, 6)]].sum(axis=1)
    d_towers = df[[f"d{i}_towers_killed" for i in range(1, 6)]].sum(axis=1)

    df["tower_advantage"] = r_towers - d_towers

    # First blood
    r_fb = df[[f"r{i}_firstblood_claimed" for i in range(1, 6)]].sum(axis=1)
    d_fb = df[[f"d{i}_firstblood_claimed" for i in range(1, 6)]].sum(axis=1)

    df["first_blood"] = r_fb - d_fb

    return df

def add_fighting_features(df):
    # Team fight participation
    for team in ['r', 'd']:
        tf_cols = [f"{team}{i}_teamfight_participation" for i in range(1, 6)]
        df[f"{team}_avg_teamfight_participation"] = df[tf_cols].mean(axis=1)

    # Stuns
    for team in ['r', 'd']:
        stuns_cols = [f"{team}{i}_stuns" for i in range(1, 6)]
        df[f"{team}_total_stuns"] = df[stuns_cols].sum(axis=1)

    # Last hits
    for team in ['r', 'd']:
        lh_cols = [f"{team}{i}_lh" for i in range(1, 6)]
        df[f"{team}_total_lh"] = df[lh_cols].sum(axis=1)

    # Last hit advantage
    df["lh_advantage"] = df["r_total_lh"] - df["d_total_lh"]

    return df

def prepare_features(df):
    df = df.copy()
    df = add_lane_dominance_features(df)
    df = add_team_features(df)
    df = add_objective_features(df)
    df = add_fighting_features(df)

    # Replace infinities with 0
    df = df.replace([np.inf, -np.inf], 0)
    
    # Fill NaN values with 0
    df = df.fillna(0)

    return df

def add_hero_features(df):
    # Create hero pair features
    radiant_heroes = df[["r1_hero_id", "r2_hero_id", "r3_hero_id", "r4_hero_id", "r5_hero_id"]].values
    dire_heroes = df[["d1_hero_id", "d2_hero_id", "d3_hero_id", "d4_hero_id", "d5_hero_id"]].values

    # Calculate hero synergies (simple counter for now)
    for i in range(5):
        for j in range(5):
            df[f"r_hero_synergy_{i}_{j}"] = radiant_heroes[:, i] * radiant_heroes[:, j]
            df[f"d_hero_synergy_{i}_{j}"] = dire_heroes[:, i] * dire_heroes[:, j]

    # Calculate counters
    for i in range(5):
        for j in range(5):
            df[f"hero_counter_{i}_{j}"] = radiant_heroes[:, i] * dire_heroes[:, j]

    return df

#end of lecture

def add_team_features(df):
    r_kills = df[[f"r{i}_kills" for i in range(1, 6)]].sum(axis=1)
    r_deaths = df[[f"r{i}_deaths" for i in range(1, 6)]].sum(axis=1)
    r_assists = df[[f"r{i}_assists" for i in range(1, 6)]].sum(axis=1)
    r_denies = df[[f"r{i}_denies" for i in range(1, 6)]].sum(axis=1)
    r_gold = df[[f"r{i}_gold" for i in range(1, 6)]].sum(axis=1)
    r_xp = df[[f"r{i}_xp" for i in range(1, 6)]].sum(axis=1)
    r_level = df[[f"r{i}_level" for i in range(1, 6)]].sum(axis=1)

    d_kills = df[[f"d{i}_kills" for i in range(1, 6)]].sum(axis=1)
    d_deaths = df[[f"d{i}_deaths" for i in range(1, 6)]].sum(axis=1)
    d_assists = df[[f"d{i}_assists" for i in range(1, 6)]].sum(axis=1)
    d_denies = df[[f"d{i}_denies" for i in range(1, 6)]].sum(axis=1)
    d_gold = df[[f"d{i}_gold" for i in range(1, 6)]].sum(axis=1)
    d_xp = df[[f"d{i}_xp" for i in range(1, 6)]].sum(axis=1)
    d_level = df[[f"d{i}_level" for i in range(1, 6)]].sum(axis=1)

    df["kills_advantage"] = r_kills - d_kills
    df["deaths_advantage"] = r_deaths - d_deaths
    df["assists_advantage"] = r_assists - d_assists
    df["denies_advantage"] = r_denies - d_denies
    df["gold_advantage"] = r_gold - d_gold
    df["xp_advantage"] = r_xp - d_xp
    df["level_advantage"] = r_level - d_level

    df["kills_ratio"] = r_kills / (d_kills + 1e-3)
    df["deaths_ratio"] = r_deaths / (d_deaths + 1e-3)
    df["assists_ratio"] = r_assists / (d_assists + 1e-3)
    df["denies_ratio"] = r_denies / (d_denies + 1e-3)
    df["gold_ratio"] = r_gold / (d_gold + 1e-3)
    df["xp_ratio"] = r_xp / (d_xp + 1e-3)
    df["level_ratio"] = r_level / (d_level + 1e-3)

    return df

In [238]:
df = add_team_features(df)
df = add_hero_features(df)
df = add_objective_features(df)

test_features = add_team_features(test_features)
test_features = add_hero_features(test_features)
test_features = add_objective_features(test_features)

In [239]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(df, test_size=0.2)

In [240]:
y_train = X_train['radiant_win']
X_train = X_train.drop(['radiant_win', 'ID'], axis=1)

y_val = X_val['radiant_win']
X_val = X_val.drop(['radiant_win', 'ID'], axis=1)

X_test = test_features.drop('ID', axis=1)

In [241]:
X_train.shape

(23740, 338)

In [147]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

In [ ]:
# for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
'''
y_val_probas = model.predict_proba(X_val)
y_val_pred = y_val_probas[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred > 0.5)
print("Validation ROC AUC:", roc_auc)
print("Validation Accuracy:", accuracy)
print('--------')
'''

In [ ]:
'''
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

# Logistic Regression model
model = LogisticRegression(C=0.10)


model = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.1,
    num_leaves=31,
    random_state=42
)

model = GradientBoostingClassifier(n_estimators=100)

model.fit(X_train, y_train)
'''

In [242]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
'''
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

def select_features(X_train, y_train, X_val, X_test):
    selector = SelectFromModel(
        RandomForestClassifier(n_estimators=100, random_state=42),
        prefit=False,
        max_features=300
    )
    selector.fit(X_train, y_train)

    X_train_selected = selector.transform(X_train)
    X_val_selected = selector.transform(X_val)
    X_test_selected = selector.transform(X_test)

    return X_train_selected, X_val_selected, X_test_selected, selector

# Example usage
X_train_selected, X_val_selected, X_test_selected, selector = select_features(X_train, y_train, X_val, X_test)
'''

In [245]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rf_selector = RandomForestClassifier(random_state=42, n_estimators=100)
rfe = RFE(estimator=rf_selector, n_features_to_select=50, step=10)
X_train = rfe.fit_transform(X_train, y_train)
X_val = rfe.transform(X_val)
X_test = rfe.transform(X_test)

In [178]:
'''
The main parameters used by a Random Forest Classifier are:

n_estimators = number of trees in the ensamble.
criterion = the function used to evaluate the quality of a split.
max_depth = maximum number of levels allowed in each tree.
max_features = maximum number of features considered when splitting a node.
min_samples_leaf = minimum number of samples which can be stored in a tree leaf.
min_samples_split = minimum number of samples necessary in a node to cause node splitting.
'''

'\nThe main parameters used by a Random Forest Classifier are:\n\ncriterion = the function used to evaluate the quality of a split.\nmax_depth = maximum number of levels allowed in each tree.\nmax_features = maximum number of features considered when splitting a node.\nmin_samples_leaf = minimum number of samples which can be stored in a tree leaf.\nmin_samples_split = minimum number of samples necessary in a node to cause node splitting.\nn_estimators = number of trees in the ensamble.\n'

In [246]:
from sklearn.model_selection import GridSearchCV

def create_rf_param_grid():
    return {
        'n_estimators': [150, 300], 
        'max_depth': [20, 40],
        'min_samples_split': [5, 10],
        'min_samples_leaf': [2, 4], 
        'max_features': ['sqrt', 'log2'], # commonly used feature selection methods
        'class_weight': [None, 'balanced'],
    } 

def perform_grid_search(X_train, y_train, param_grid=None, cv=3, n_jobs=-1):
    if param_grid is None:
        param_grid = create_rf_param_grid()

    base_model = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        cv=cv,
        scoring='roc_auc',
        verbose=2,
        n_jobs=n_jobs
    )

    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters found: {grid_search.best_params_}")
    print(f"Best ROC-AUC score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_, grid_search


In [247]:
model, grid_search = perform_grid_search(X_train, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
Best parameters found: {'class_weight': 'balanced', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 300}
Best ROC-AUC score: 0.8184


In [248]:
from sklearn.metrics import roc_auc_score, accuracy_score

y_val_probas = model.predict_proba(X_val)
y_val_pred = y_val_probas[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred > 0.5)
print("Validation ROC AUC:", roc_auc)
print("Validation Accuracy:", accuracy)
print('--------')

Validation ROC AUC: 0.8168260736354122
Validation Accuracy: 0.7277169334456614
--------


In [249]:
print("Best Params:", grid_search.best_params_)

Best Params: {'class_weight': 'balanced', 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 300}


In [250]:
y_test_pred = model.predict_proba(X_test)[:, 1]

In [251]:
y_test_pred.shape

(10000,)

In [252]:
y_test_pred

array([0.29334029, 0.4132065 , 0.61063144, ..., 0.05895948, 0.6881677 ,
       0.74359689])

In [253]:
import numpy as np
submission_array = np.vstack([test_features['ID'].values, y_test_pred]).T
submission_df = pd.DataFrame(submission_array, columns=['ID', 'radiant_win']).reset_index(drop=True)
submission_df.to_csv('final_submission_during_class.csv', index=None)